In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


# Collecting Dataset

In [2]:
DATASET_DIR = '../datasets/UCF101/'

In [3]:
df = pd.read_csv(DATASET_DIR+ 'train.csv')
df['clip_path'] = df['clip_path'].apply(lambda x: DATASET_DIR + x) 
df.head()

,clip_name,clip_path,label
0,v_Swing_g05_c02,../datasets/UCF101//train/Swing/v_Swing_g05_c0...,Swing
1,v_Swing_g21_c03,../datasets/UCF101//train/Swing/v_Swing_g21_c0...,Swing
2,v_Swing_g07_c01,../datasets/UCF101//train/Swing/v_Swing_g07_c0...,Swing
3,v_Swing_g24_c04,../datasets/UCF101//train/Swing/v_Swing_g24_c0...,Swing
4,v_Swing_g20_c03,../datasets/UCF101//train/Swing/v_Swing_g20_c0...,Swing


In [4]:
df['label'].unique()

array(['Swing', 'SkyDiving', 'BreastStroke', 'TableTennisShot', 'FloorGymnastics', 'Rowing', 'SoccerJuggling', 'BenchPress', 'TennisSwing', 'FrisbeeCatch', 'Kayaking', 'ApplyLipstick', 'RopeClimbing', 'JumpingJack', 'BlowingCandles', 'BabyCrawling', 'HammerThrow', 'BoxingPunchingBag', 'Skiing', 'UnevenBars',
       'PullUps', 'RockClimbingIndoor', 'HeadMassage', 'HulaHoop', 'Rafting', 'ShavingBeard', 'LongJump', 'FieldHockeyPenalty', 'VolleyballSpiking', 'Punch', 'Archery', 'PlayingDhol', 'CleanAndJerk', 'PlayingDaf', 'GolfSwing', 'PlayingSitar', 'IceDancing', 'SkateBoarding', 'BodyWeightSquats', 'HandstandPushups',
       'BalanceBeam', 'WallPushups', 'HorseRiding', 'Bowling', 'JumpRope', 'CuttingInKitchen', 'CliffDiving', 'PlayingGuitar', 'FrontCrawl', 'HandstandWalking', 'Fencing', 'Basketball', 'Typing', 'TaiChi', 'PlayingCello', 'Lunges', 'Shotput', 'Nunchucks', 'ApplyEyeMakeup', 'BaseballPitch', 'Mixing', 'PlayingTabla',
       'BandMarching', 'MoppingFloor', 'PushUps', 'Military

In [5]:
punch_df = df[df['label']=='Punch'].reset_index(drop=True)
box_p_bag_df = df[df['label']=='BoxingPunchingBag'].reset_index(drop=True)
walking_df = df[df['label']=='WalkingWithDog'].reset_index(drop=True)

In [6]:
train_dir =  list(walking_df['clip_path']) + list(box_p_bag_df['clip_path']) + list(punch_df['clip_path'])
train_label = list( walking_df['label']) + list(box_p_bag_df['label']) + list(punch_df['label'])


len(train_dir)

334

# Infrence on Dataset

In [7]:
from ultralytics import YOLO
import cv2

In [8]:
model = YOLO('models/yolov8s-pose.pt') 

In [9]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=100)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()
for train_df in [walking_df, box_p_bag_df]:  #, punch_df[0:1]
    label = train_df['label'][0]
    print(f"Training: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': [label]*len(batch)})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

Training: WalkingWithDog


Processing Videos:   0%|          | 0/92 [00:05<?, ?it/s]


KeyboardInterrupt: 

### testing reshape

In [ ]:
batch = batch

In [ ]:
batch.shape

(1, 40, 34)

In [ ]:
t = batch.reshape((-1, 1 , 20, 34))

In [ ]:
t.shape

(2, 1, 20, 34)

In [ ]:
[label]*len(batch)

['WalkingWithDog', 'WalkingWithDog']

### debug

In [ ]:
np.array(keypoints_df['keypoints'].tolist()).shape

(2, 20, 34)

# convert to datset

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/keypoints_df_10frames_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)

In [ ]:
keypoints_df['label'].value_counts()

label
BoxingPunchingBag    1210
WalkingWithDog        900
Name: count, dtype: int64

# Custom Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os

from ultralytics import YOLO
import cv2
import pandas as pd

In [ ]:
C_DATASET_DIR = 'examples/videos/'

In [ ]:
model = YOLO('models/yolov8n-pose.pt') 

In [ ]:
walking_df =  pd.DataFrame({'clip_path': [C_DATASET_DIR+'Walking.mp4'], 'label': ['walking']})
punching_df =  pd.DataFrame({'clip_path': [C_DATASET_DIR+'Punching.mp4'], 'label': ['punching']})

In [ ]:
vp = VideoProcessor(model, max_frames=1000)

keypoints_df = pd.DataFrame()
for train_df in [walking_df, punching_df]:  #
    class_keypoints_df = pd.DataFrame()
    label = train_df['label'][0]
    print(f"Training: {label}")
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1, 20, 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': [label]*len(batch)})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])    
    

cv2.destroyAllWindows()

Training: walking


Processing Videos: 100%|██████████| 1/1 [00:31<00:00, 31.37s/it]


Training: punching


Processing Videos: 100%|██████████| 1/1 [00:32<00:00, 32.09s/it]


In [ ]:
# SAVE DATASET

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/custom_dataset/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)

In [ ]:
keypoints_df['label'].value_counts()

label
walking     50
punching    50
Name: count, dtype: int64